In [1]:
from google.colab import drive 
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import time

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader, Dataset
import matplotlib.pyplot as plt


In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
def To3DFeatures(array):
    new_array = []

    for row in array:
        frame_list = []
        one_frame = []
        joint_coordinate = []
        for index, item in enumerate(row):
          joint_coordinate.append(item)
          if index % 3 == 2:
            one_frame.append(joint_coordinate)
            joint_coordinate = []
          if index % 60 == 59:
              frame_list.append(one_frame)
              one_frame = []
        new_array.append(frame_list)
    
    return new_array

def ToMotionFeature(array):
    motion_feature = []
    
    for index1, frame_list in enumerate(array):
        frame_motion = []
        for index2, frame in enumerate(frame_list):
            if index2 > 0:
                motion = []
                for index3, joint in enumerate(frame):
                    
                    x_movement = frame_list[index2][index3][0] - frame_list[index2 - 1][index3][0]
                    y_movement = frame_list[index2][index3][1] - frame_list[index2 - 1][index3][1]
                    z_movement = frame_list[index2][index3][2] - frame_list[index2 - 1][index3][2]
                    motion.append([x_movement, y_movement, z_movement])
                frame_motion.append(motion)   
                
                
        motion_feature.append(frame_motion)
    return torch.FloatTensor(motion_feature)

def DataSplit(data, r1 = 5, r2 = 10, split_size = 0.2):
  #input: loaded data,
  #output: train, validation, test set (np array)
  #explanation: remove the first column (id), feature set is the column from 1 - 960, test set is the column 961

  features = [i for i in range(1,961)]
  labels = [961]
  
  x_train = data[features]
  y_train = data[labels]

  x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size = split_size, random_state = r1)
  x_validation, x_test, y_validation, y_test = train_test_split(x_test, y_test, test_size = 0.5, random_state = r2)


  return x_train.values, x_validation.values, x_test.values, y_train.values, y_validation.values, y_test.values


In [4]:
train_path = '/content/drive/MyDrive/Project1-Group 46/train.csv'
test_path = '/content/drive/MyDrive/Project1-Group 46/test.csv'

train_data = pd.read_csv(train_path, header = None)


x_train, x_validation, x_test, y_train, y_validation, y_test = DataSplit(train_data)



x_train = To3DFeatures(x_train)
x_validation = To3DFeatures(x_validation)
x_test = To3DFeatures(x_test)



x_train = torch.FloatTensor(x_train)
x_validation = torch.FloatTensor(x_validation)
x_test = torch.FloatTensor(x_test)
y_train = torch.LongTensor(y_train)
y_validation = torch.LongTensor(y_validation)
y_test = torch.LongTensor(y_test)

y_train = torch.flatten(y_train)
y_validation = torch.flatten(y_validation)
y_test = torch.flatten(y_test)



x_train_motion = ToMotionFeature(x_train)
x_validation_motion = ToMotionFeature(x_validation)
x_test_motion = ToMotionFeature(x_test)



x_train = np.swapaxes(x_train,1,3)
x_validation = np.swapaxes(x_validation,1,3)
x_test = np.swapaxes(x_test,1,3)

x_train_motion = np.swapaxes(x_train_motion,1,3)
x_validation_motion = np.swapaxes(x_validation_motion,1,3)
x_test_motion = np.swapaxes(x_test_motion,1,3)



In [5]:
class SKEDataset(Dataset):

    def __init__(self, x1_train, x2_train, y_train):

        self.x1_train = x1_train
        self.x2_train = x2_train

        self.y_train = y_train

    def __len__(self):
        return len(self.x1_train)

    def __getitem__(self, index):


        return self.x1_train[index], self.x2_train[index], self.y_train[index]


train_set = SKEDataset(x_train, x_train_motion, y_train)
dev_set = SKEDataset(x_validation, x_validation_motion, y_validation)


train_loader = DataLoader(train_set, batch_size = 64)
dev_loader = DataLoader(dev_set, batch_size = 64)

In [6]:
class CNN2d_x1(nn.Module):
  def __init__(self):
    super(CNN2d_x1, self).__init__()

    #20 * 16 * 3
    self.conv1 = nn.Sequential(
        nn.Conv2d(in_channels=3, 
                  out_channels=16, 
                  kernel_size=3, 
                  stride = 1,
                  padding = 1
        ),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2,stride = 2)
    )

    #16 * 10 * 8  


  def forward(self, x):
    x = self.conv1(x)

    return x


class CNN2d_x2(nn.Module):
  def __init__(self):
    super(CNN2d_x2, self).__init__()

    # 3 * 20 * 15
    self.conv1 = nn.Sequential(
        nn.Conv2d(in_channels=3, 
                  out_channels=16, 
                  kernel_size=3, 
                  stride = 1,
                  padding = 1
        ),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2,stride = 2)
    )

    #16 * 10 * 7 

  def forward(self, x):
    x = self.conv1(x)

    return x

class Ensemble(nn.Module):
    def __init__(self, modelA, modelB):
        super(Ensemble, self).__init__()
    
        self.modelA = modelA
        self.modelB = modelB
        
        self.fc1 = nn.Linear(16 * 10 * 15, 50)
    
    
        #16 * 10 * 7 
    
    def forward(self, x1, x2):
      x1 = self.modelA(x1)
      x2 = self.modelB(x2)
      
      x = torch.cat((x1,x2), dim = 3)
      x = x.view(-1, 16 * 10 * 15)
      
      x = self.fc1(x)
      
      return F.log_softmax(x, dim=1)


In [7]:

def train(model, criterion, opti, train_loader, dev_loader, max_eps):
  best_acc = 0
  st = time.time()

  train_correct = []
  dev_correct = []
  train_losses = []
  dev_losses = []
  for ep in range(max_eps):
    trn_corr = 0
    tst_corr = 0
    total_y = 0
    count = 0
    mean_loss = 0

    for b, (x1_train, x2_train, y_train) in enumerate(train_loader):
      b += 1
      y_pred = model(x1_train, x2_train)
      loss = criterion(y_pred, y_train)

      predicted = torch.max(y_pred.data, 1)[1]
      batch_corr = (predicted == y_train).sum()
      trn_corr += batch_corr

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      if b % 30 == 0:
        print(("Iteration {} of epoch {} complete. Loss: {}; Acc =: {}, Time taken (s): {}".format(b, ep, loss.item(), batch_corr/len(y_train), (time.time()-st))))
    

    with torch.no_grad():
      for b,(x1_dev, x2_dev, y_dev) in enumerate(dev_loader):
        y_val = model(x1_dev, x2_dev)
        predicted = torch.max(y_val.data,1)[1]
        tst_corr += (predicted == y_dev).sum()
        total_y += len(predicted)
        count += 1

        mean_loss += criterion(y_val, y_dev)
    
    acc_dev = tst_corr / total_y
    mean_loss = mean_loss / count

    print("Epoch {} complete! Development Accuracy: {}; Development Loss: {}".format(ep, acc_dev, mean_loss))
    if acc_dev > best_acc:
        print("Best development accuracy improved from {} to {}, saving model...".format(best_acc, acc_dev))
        best_acc = acc_dev
        torch.save(model.state_dict(), 'sstcls_{}.dat'.format(ep))


In [ ]:
model_a = CNN2d_x1()
model_b = CNN2d_x2()

ensemble_model = Ensemble(model_a, model_b)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(ensemble_model.parameters(), lr = 0.01)
max_eps = 100
train(ensemble_model, criterion, optimizer, train_loader, dev_loader, max_eps)


In [18]:
pre_model_a = CNN2d_x1()
pre_model_b = CNN2d_x2()

predict_model = Ensemble(pre_model_a, pre_model_b)
predict_model.load_state_dict(torch.load('sstcls_53.dat'))

with torch.no_grad():
  y_eval = predict_model.forward(x_test, x_test_motion)
  loss = criterion(y_eval, y_test)
  prediction = torch.max(y_eval.data,1)[1]

In [19]:
accuracy_score(y_test, prediction)

0.3588924387646432